<a href="https://colab.research.google.com/github/nehadubey1205/ResearchWork/blob/main/New_Approach_HateandBullySent30April.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/combined_file_New.csv')

In [ ]:
data.tail(2)

In [ ]:
# Print the first three rows of the DataFrame
data.iloc[5900:6200]

In [ ]:
# we want to remove 6084
data= data.drop(index=[6084])

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

LSTM

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:

pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
     


from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
X = data['Processed_Tweets'].values
y = data['Bully_Label'].values
     

from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
     


X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=100)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Creating the deep learning model
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

In [ ]:
#from keras.callbacks import EarlyStopping
#from keras.utils import to_categorical

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(X_train, to_categorical(y_train), epochs=10, batch_size=64, validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.01)])

In [ ]:
y_predict = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
# Evaluating the performance of the model

accuracy = accuracy_score(y_test, y_predict)

In [ ]:

precision = precision_score(y_test, y_predict, average='weighted')
recall = recall_score(y_test, y_predict, average='weighted')
f1 = f1_score(y_test, y_predict, average='weighted')
     

#print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

LSTM -2 **LSTM second Methode**

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load the dataset
df = pd.read_csv("/content/combined_file_New.csv")

In [ ]:
df.tail()

In [ ]:
df= df.drop(index=[6084])

In [ ]:
# Remove unwanted columns and clean the text
df = df[['Processed_Tweets', 'Bully_Label']]
df['Processed_Tweets'] = df['Processed_Tweets'].apply(lambda x: re.sub(r'http\S+', '', x))
df['Processed_Tweets'] = df['Processed_Tweets'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
df['Processed_Tweets'] = df['Processed_Tweets'].apply(lambda x: x.lower())

In [ ]:
import nltk
nltk.download('popular')

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Remove stop words
stop_words = set(stopwords.words('english'))
train_df['Processed_Tweets'] = train_df['Processed_Tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
test_df['Processed_Tweets'] = test_df['Processed_Tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
# Convert the labels to numerical values
train_df['Bully_Label'] = train_df['Bully_Label'].astype('category').cat.codes
test_df['Bully_Label'] = test_df['Bully_Label'].astype('category').cat.codes


In [ ]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = vectorizer.fit_transform(train_df['Processed_Tweets'])

# Transform the test data using the fitted vectorizer
X_test = vectorizer.transform(test_df['Processed_Tweets'])

In [ ]:
# Convert the labels to numpy arrays
y_train = np.array(train_df['Bully_Label'])
y_test = np.array(test_df['Bully_Label'])

In [ ]:
# Convert the text data to sequences of integers
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_df['Processed_Tweets'])

X_train_seq = tokenizer.texts_to_sequences(train_df['Processed_Tweets'])
X_test_seq = tokenizer.texts_to_sequences(test_df['Processed_Tweets'])

# Pad the sequences to a fixed length
X_train_seq = pad_sequences(X_train_seq, maxlen=500)
X_test_seq = pad_sequences(X_test_seq, maxlen=500)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=500))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

# Train the model
model.fit(X_train_seq, y_train, batch_size=32, epochs=10, validation_data=(X_test_seq, y_test))


In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_seq, y_test, verbose=0)
print("Test accuracy:", accuracy)


In [ ]:
from sklearn.metrics import classification_report

# Make predictions on the test data
y_pred = model.predict(X_test_seq)
y_pred = (y_pred > 0.5).astype(int)

# Print the precision, recall, and F1 score
print(classification_report(y_test, y_pred))


Voting Classifiers

In [ ]:
df = pd.read_csv('/content/combined_file_New.csv')

In [ ]:
# we want to remove 6084
df= df.drop(index=[6084])

In [ ]:
df.tail(2)

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:


import nltk
nltk.download('popular')

In [ ]:
from nltk.corpus import stopwords

# Download Hinglish stop words
nltk.download('stopwords-indian')
     

nltk.download('stopwords')

In [ ]:
# Preprocessing the dataset
vectorizer = TfidfVectorizer(stop_words='english')
     

X = vectorizer.fit_transform(df['Processed_Tweets'])
y = df['Bully_Label']

In [ ]:
# Creating base models
model1 = DecisionTreeClassifier()
model2 = SVC(kernel='linear', probability=True)
model3 = MultinomialNB()

In [ ]:
# Creating an ensemble model
ensemble = VotingClassifier(estimators=[('dt', model1), ('svm', model2), ('nb', model3)], voting='soft')

# Training the ensemble model
ensemble.fit(X, y)

In [ ]:
# Evaluating the performance of the model
y_pred = ensemble.predict(X)
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred, average='weighted')
recall = recall_score(y, y_pred, average='weighted')
f1 = f1_score(y, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

#BERT Model

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
# Load the dataset from the CSV file
df = pd.read_csv('/content/combined_file_New.csv')
# we want to remove 6084
df= df.drop(index=[6084])


In [ ]:
# Randomize the rows
#df= df.sample(frac=1)

In [ ]:
df.tail(10)

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support



# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

# Load the BERT model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Tokenize the text messages
train_encodings = tokenizer(list(train_df['Processed_Tweets']), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df['Processed_Tweets']), truncation=True, padding=True)

# Convert the labels to numerical values
train_labels = np.array(list(train_df['Bully_Label'])).astype(int)
test_labels = np.array(list(test_df['Bully_Label'])).astype(int)

# Create a PyTorch dataset from the encoded data
class CyberbullyingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Define a function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:


# Create a Trainer object
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=CyberbullyingDataset(train_encodings, train_labels),
    eval_dataset=CyberbullyingDataset(test_encodings, test_labels),
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


In [ ]:
# Evaluate the model on the test set
eval_result = trainer.evaluate()

# Print the results
print(f"Accuracy: {eval_result['eval_accuracy']}")
print(f"F1 Score: {eval_result['eval_f1']}")
print(f"Precision: {eval_result['eval_precision']}")
print(f"Recall: {eval_result['eval_recall']}")